In [1]:
import torch
from src.common import Dataset, RavenMode
from src.specLoader import get_specification, get_std
from src.netLoader import get_net
from src.adaptiveRavenBackend import AdaptiveRavenBackend
from src.adaptiveRavenResult import AdaptiveRavenResultList
from raven.src.config import mnist_data_transform
import raven.src.config as config
from auto_LiRPA.operators import BoundLinear, BoundConv
import numpy as np
from raven.src.network_conversion_helper import get_pytorch_net
from auto_LiRPA import BoundedModule, PerturbationLpNorm, BoundedTensor
from src.multinet_gurobi_certifier import MultiNetMILPTransformer

from network_utils import *

/apps/conda/cmxu/envs/autolipra/lib/python3.11/site-packages/torch/utils/cpp_extension.py:28: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import packaging  # type: ignore[attr-defined]
No CUDA runtime is found, using CUDA_HOME='/usr'


In [2]:
dataset = Dataset.CIFAR10
prop_count = 100
count_per_prop = 1
total_input_count = prop_count * count_per_prop
eps = 8/255
net_names = ['test']
nets = ['project_networks/cifar10_cnn_2layer_w1.onnx']
nets = [load_onnx_to_pytorch(a) for a in nets]
images, labels, constraint_matrices, lbs, ubs = get_specification(dataset=dataset,
                                                            raven_mode=RavenMode.UAP, 
                                                            count=total_input_count, nets=nets, eps=eps,
                                                            dataloading_seed=1232,
                                                            net_names=net_names,
                                                            only_cutoff=False)

Files already downloaded and verified
eps tensor tensor([0.1551, 0.1573, 0.1561])


In [3]:
class Property:
    def __init__(self, inputs, labels, eps, constraint_matrices, lbs, ubs) -> None:
        self.inputs = inputs
        self.labels = labels
        self.eps = eps
        self.constraint_matrices = constraint_matrices
        self.lbs = lbs
        self.ubs = ubs

In [4]:
def get_linear_coefs_biases(model, prop, device, input_name, final_name):
    ptb = PerturbationLpNorm(norm = np.inf, x_L=prop.lbs, x_U=prop.ubs)
    bounded_images = BoundedTensor(prop.inputs, ptb)
    coef_dict = {final_name: [input_name]}
    result = model.compute_bounds(x=(bounded_images,), method='CROWN-Optimized', C=prop.constraint_matrices,
                                    bound_upper=False, return_A=True, needed_A_dict=coef_dict, 
                                    multiple_execution=False, execution_count=None, ptb=ptb, 
                                    unperturbed_images = prop.inputs)
    lower_bnd, upper, A_dict = result
    lA = A_dict[final_name][input_name]['lA']
    lbias = A_dict[final_name][input_name]['lbias']
    lA = torch.reshape(lA,(1, 9,-1))
    return lA, lbias, lower_bnd

In [5]:
def populate_names(model):
    final_names = []
    input_names = []
    layer_names = []
    i = 0
    last_name = None
    for node_name, node in model._modules.items():
        if i == 0:
            input_names.append(node_name)
        i += 1
        if type(node) in [BoundLinear, BoundConv]:
            layer_names.append(node_name)
            last_name = node_name
    assert last_name is not None
    final_names.append(node_name)
    return input_names, layer_names, final_names

In [19]:
torch.nn.Sequential(*list(nets[0].modules())[1:])

Sequential(
  (0): Conv2d(3, 4, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(4, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): Linear(in_features=512, out_features=128, bias=True)
  (6): ReLU()
  (7): Linear(in_features=128, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=128, bias=True)
  (10): ReLU()
  (11): Linear(in_features=128, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=128, bias=True)
  (14): ReLU()
  (15): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
def g2s(model):
    return torch.nn.Sequential(*list(nets[0].modules())[1:])

GraphModule(
  (0/Conv): Conv2d(3, 4, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1/Relu): ReLU()
  (2/Conv): Conv2d(4, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3/Relu): ReLU()
  (4/Flatten): Flatten(start_dim=1, end_dim=-1)
  (5/Gemm): Linear(in_features=512, out_features=128, bias=True)
  (6/Relu): ReLU()
  (7/Gemm): Linear(in_features=128, out_features=128, bias=True)
  (8/Relu): ReLU()
  (9/Gemm): Linear(in_features=128, out_features=128, bias=True)
  (10/Relu): ReLU()
  (11/Gemm): Linear(in_features=128, out_features=128, bias=True)
  (12/Relu): ReLU()
  (13/Gemm): Linear(in_features=128, out_features=128, bias=True)
  (14/Relu): ReLU()
  (15/Gemm): Linear(in_features=128, out_features=10, bias=True)
)

In [7]:
for i in range(prop_count):
    start = i * count_per_prop
    end = start + count_per_prop
    prop_images, prop_labels, prop_constraint_matrices = images[start:end], labels[start:end], constraint_matrices[start:end]
    prop_lbs, prop_ubs = lbs[start:end], ubs[start:end]
    prop = Property(inputs=prop_images, labels=prop_labels, 
                    eps=eps / get_std(dataset=dataset, transform=True),
                    constraint_matrices=prop_constraint_matrices, lbs=prop_lbs, ubs=prop_ubs)
    final_names = []
    input_names = []
    models = []
    for net in nets:
        torch_net = get_pytorch_net(model=net, remove_last_layer=False, all_linear=False)
        models.append(BoundedModule(torch_net, (prop.inputs), bound_opts={}))
    for net in models:
        ins, _, fns = populate_names(net)
        final_names.append(fns[0])
        input_names.append(ins[0])
    get_linear_coefs_biases(models[0], prop)
    break

TypeError: 'GraphModule' object is not iterable